In [1]:
#pybullet is a simple Python interface to the physics engine Bullet.
import pybullet

In [2]:
#Starting the simulation is dead simple
pybullet.connect(pybullet.GUI)
# without GUI: pybullet.connect(pybullet.DIRECT)

0

In [3]:
#You can reset the simulation to its original state whenever you want which makes it appealing for all kind of reinforcement learning algorithms.
#After resetting the environment, all models have to be recreated. 
pybullet.resetSimulation()

In [4]:
#pybullet can load kinematic descriptions of robots or other objects from (URDF) unified robotics description format files.
#You could also create your robot manually with a bunch of function calls but you usually don't want that because there are so many tools that support URDF
#for example, inverse kinematics solvers, visualization tools, etc. Other supported formats are Bullet's own format, 
#Gazebo's SDF, and MuJoCo's MJCF files. These support multiple objects and allow you to load entire simulation scenarios at once.
#pybullet also comes with some objects that are often useful, for example, a plane:
import pybullet_data
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())

In [5]:
#Let's load some more complicated object: ROS-Industrial has some URDFs for Kuka robots.
# Load the URDF file (replace 'path_to_urdf_file.urdf' with your URDF file path)
robot_id = pybullet.loadURDF('arm.urdf', basePosition=[0, 0, 0], baseOrientation=[0, 0, 0, 1])

In [6]:
#Orientation is usually given in quaternions (x, y, z, w).
position, orientation = pybullet.getBasePositionAndOrientation(robot_id)
orientation

(0.0, 0.0, 0.0, 1.0)

In [7]:
pybullet.getNumJoints(robot_id) # get fixed and revolute joints

6

In [8]:
#We can request information about each joint.
joint_index = 2
joint_info = pybullet.getJointInfo(robot_id, joint_index)
name, joint_type, lower_limit, upper_limit = \
    joint_info[1], joint_info[2], joint_info[8], joint_info[9]
name, joint_type, lower_limit, upper_limit


(b'arm_joint_2', 0, -1.57079632679, 1.57079632679)

In [9]:
# request the current state of each joint returned by pybullet.getJointInfo(...)
joint_positions = [j[0] for j in pybullet.getJointStates(robot_id, range(6))]
joint_positions

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [10]:
#ask for the current position of a link.
world_position, world_orientation = pybullet.getLinkState(robot_id, 2)[:2]
world_position

(0.0, 0.0, 0.33599999999999997)

In [11]:
#set up the simulation
pybullet.setGravity(0, 0, -9.81)   # everything should fall down
pybullet.setTimeStep(0.0001)       # this slows everything down, but let's be accurate...
pybullet.setRealTimeSimulation(0)  # we want to be faster than real time :)

In [12]:
#give the robot something to do. We will set the desired joint angle. There are other control modes: velocity control and torque control.
pybullet.setJointMotorControlArray(
    robot_id, range(6), pybullet.POSITION_CONTROL,
    targetPositions=[0.1] * 6)

In [13]:
#can step through the simulation
for _ in range(10000):
    pybullet.stepSimulation()

In [14]:
# The robot moves, but it will almost fall over. What happened? Well, we forgot to attach it to the ground! Let's do that now:
pybullet.resetSimulation()
plane = pybullet.loadURDF("plane.urdf")
robot = pybullet.loadURDF("arm.urdf",
                          [0, 0, 0], useFixedBase=1)  # use a fixed base!
pybullet.setGravity(0, 0, -9.81)
pybullet.setTimeStep(0.0001)
pybullet.setRealTimeSimulation(0)

In [15]:
pybullet.setJointMotorControlArray(
    robot, range(6), pybullet.POSITION_CONTROL,
    targetPositions=[0.1] * 6)
for _ in range(10000):
    pybullet.stepSimulation()